In [ ]:
import math
import os
import shutil
from collections import Counter

def reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
                   valid_ratio):
    # 读取训练数据标签。
    with open(os.path.join(data_dir, label_file), 'r') as f:
        # 跳过文件头行（栏名称）。
        lines = f.readlines()[1:]
        tokens = [l.rstrip().split(',') for l in lines]
        idx_label = dict(((idx, label) for idx, label in tokens))
    labels = set(idx_label.values())

    num_train = len(os.listdir(os.path.join(data_dir, train_dir)))
    # 训练集中数量最少一类的狗的数量。
    min_num_train_per_label = (
        Counter(idx_label.values()).most_common()[:-2:-1][0][1])
    # 验证集中每类狗的数量。
    num_valid_per_label = math.floor(min_num_train_per_label * valid_ratio)
    label_count = dict()

    def mkdir_if_not_exist(path):
        if not os.path.exists(os.path.join(*path)):
            os.makedirs(os.path.join(*path))

    # 整理训练和验证集。
    for train_file in os.listdir(os.path.join(data_dir, train_dir)):
        idx = train_file.split('.')[0]
        label = idx_label[idx]
        mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label])
        shutil.copy(os.path.join(data_dir, train_dir, train_file),
                    os.path.join(data_dir, input_dir, 'train_valid', label))
        if label not in label_count or label_count[label] < num_valid_per_label:
            mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            mkdir_if_not_exist([data_dir, input_dir, 'train', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'train', label))

    # 整理测试集。
    mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
    for test_file in os.listdir(os.path.join(data_dir, test_dir)):
        shutil.copy(os.path.join(data_dir, test_dir, test_file),
                    os.path.join(data_dir, input_dir, 'test', 'unknown'))

In [ ]:
data_dir = '/home/yansheng/kaggle_dog'
train_dir = 'train'
label_file = 'labels.csv'
test_dir = 'test'
input_dir = 'train_valid_test'

batch_size = 128
valid_ratio = 0.1
reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,valid_ratio)

In [1]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
import numpy as np

def transform_train(data, label):
    im = image.imresize(data.astype('float32') / 255, 224, 224)
    auglist = image.CreateAugmenter(data_shape=(3, 224, 224), resize=0, 
                        rand_crop=False, rand_resize=False, rand_mirror=True,
                        mean=np.array([0.485, 0.456, 0.406]), std=np.array([0.229, 0.224, 0.225]), 
                        brightness=0, contrast=0, 
                        saturation=0, hue=0, 
                        pca_noise=0, rand_gray=0, inter_method=2)
    for aug in auglist:
        im = aug(im)
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

def transform_test(data, label):
    im = image.imresize(data.astype('float32') / 255, 224, 224)
    auglist = image.CreateAugmenter(data_shape=(3, 224, 224), 
                        mean=np.array([0.485, 0.456, 0.406]), 
                        std=np.array([0.229, 0.224, 0.225]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

In [2]:
data_dir = '/home/yansheng/kaggle_dog'
train_dir = 'train'
label_file = 'labels.csv'
test_dir = 'test'
input_dir = 'train_valid_test'

batch_size = 128

input_str = data_dir + '/' + input_dir + '/'

# 读取原始图像文件。flag=1说明输入图像有三个通道（彩色）。
train_ds = vision.ImageFolderDataset(input_str + 'train', flag=1,
                                     transform=transform_train)
valid_ds = vision.ImageFolderDataset(input_str + 'valid', flag=1,
                                     transform=transform_test)
train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid',
                                           flag=1, transform=transform_train)
test_ds = vision.ImageFolderDataset(input_str + 'test', flag=1,
                                     transform=transform_test)

loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep')
valid_data = loader(valid_ds, batch_size, shuffle=True, last_batch='keep')
train_valid_data = loader(train_valid_ds, batch_size, shuffle=True,
                          last_batch='keep')
test_data = loader(test_ds, batch_size, shuffle=False, last_batch='keep')

# 交叉熵损失函数。
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [3]:
from mxnet.gluon import nn
from mxnet import nd

def Classifier(num_out):
    net = nn.HybridSequential()
    net.add(nn.Dense(512, activation='relu'))
    net.add(nn.Dropout(.5))
    net.add(nn.Dense(num_out))
    return net

def get_net(ctx):
    pretrained_model = gluon.model_zoo.vision.resnet50_v2(pretrained=True)
    feature_net = pretrained_model.features
    feature_net.collect_params().setattr('grad_req', 'null')
    num_outputs = 120
    net = nn.HybridSequential()
    with net.name_scope():
        net.add(feature_net)
        net.add(Classifier(num_outputs))
        net[1].collect_params().initialize(init=mx.init.Xavier())
    net.collect_params().reset_ctx(ctx)
    return net

In [4]:
import datetime
import mxnet as mx

def get_loss(data, net, ctx):
    loss = 0.0
    for feas, label in data:
        label = label.as_in_context(ctx)
        output = net(feas.as_in_context(ctx))
        cross_entropy = softmax_cross_entropy(output, label)
        loss += nd.mean(cross_entropy).asscalar()
    return loss / len(data)


def accuracy(output, label):
     return nd.mean(output.argmax(axis=1)==label).asscalar()
def evaluate_accuracy(net, data_iter, ctx=mx.cpu()):
    step = len(data_iter)
    acc = 0
    loss = 0
    for data, label in data_iter:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        output_loss = softmax_cross_entropy(output, label)
        acc += accuracy(output, label)
        loss += nd.mean(output_loss).asscalar()
    return loss / step, acc / step

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_period,
          lr_decay):
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9,
                                      'wd': wd})
    prev_time = datetime.datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0.0
        train_acc = 0.0
        if epoch > 0 and epoch % lr_period == 0:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
            train_acc += accuracy(output, label)
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss, valid_acc = evaluate_accuracy(net, valid_data, ctx)
            epoch_str = ("Epoch %d. Train loss: %.4f, Train Acc: %.4f, Valid loss %.4f, Valid Acc: %.4f, "
                         % (epoch, train_loss / len(train_data), 100 * train_acc / len(train_data),  
                            valid_loss, 100 * valid_acc))
        else:
            epoch_str = ("Epoch %d. Train loss: %f, "
                         % (epoch, train_loss / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))

In [5]:
import mxnet as mx
ctx = mx.gpu()
num_epochs = 60
learning_rate = 0.01
weight_decay = 5e-4
lr_period = 30
lr_decay = 0.5

net = get_net(ctx)
net.collect_params().reset_ctx(ctx)
net.hybridize()
train(net, train_data, valid_data, num_epochs, learning_rate,
      weight_decay, ctx, lr_period, lr_decay)

Epoch 0. Train loss: 3.9743, Train Acc: 16.5479, Valid loss 2.2622, Valid Acc: 61.1086, Time 00:01:45, lr 0.01
Epoch 1. Train loss: 1.9775, Train Acc: 51.4922, Valid loss 1.0434, Valid Acc: 78.6480, Time 00:01:55, lr 0.01
Epoch 2. Train loss: 1.2745, Train Acc: 65.0678, Valid loss 0.7323, Valid Acc: 83.8639, Time 00:01:51, lr 0.01
Epoch 3. Train loss: 1.0462, Train Acc: 70.4196, Valid loss 0.6241, Valid Acc: 85.4974, Time 00:01:53, lr 0.01
Epoch 4. Train loss: 0.9138, Train Acc: 74.0666, Valid loss 0.5655, Valid Acc: 85.8880, Time 00:01:53, lr 0.01
Epoch 5. Train loss: 0.8365, Train Acc: 75.9121, Valid loss 0.5047, Valid Acc: 87.2374, Time 00:01:53, lr 0.01
Epoch 6. Train loss: 0.7748, Train Acc: 76.9522, Valid loss 0.4657, Valid Acc: 88.0144, Time 00:01:51, lr 0.01
Epoch 7. Train loss: 0.7403, Train Acc: 78.3340, Valid loss 0.4325, Valid Acc: 88.2587, Time 00:01:52, lr 0.01
Epoch 8. Train loss: 0.6920, Train Acc: 79.2745, Valid loss 0.4045, Valid Acc: 89.0754, Time 00:01:54, lr 0.01
E